In [1]:


#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle

# If you would prefer to load the '.h5' example file rather than the '.pickle' example file. You need the deepdish package
# import deepdish as dd 

#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

#Import decoder functions
from Neural_Decoding.decoders import DenseNNDecoder


#Import hyperparameter optimization packages
#If either are not installed, give a warning
try:
    from bayes_opt import BayesianOptimization
except ImportError:
    print("\nWARNING: BayesianOptimization package is not installed. You will be unable to use section 4.")
    pass


In [32]:
###Load Data###
folder = 'input_data' #ENTER THE FOLDER THAT YOUR DATA IS IN
anm = 'JEB7'
date = '2021-04-30'

data=io.loadmat(folder + '/' + anm + '_' + date + '.mat')

N = data['gpfadat'] # (time,trials,factors) of gpfa latent trajectories
M = data['kindat']  # (time,trials,features) of kinematic features reduced with PCA

In [33]:
nTrials = N.shape[1]

idx = np.arange(0,nTrials)

randidx = np.random.choice(idx, size=(nTrials,1), replace=False).flatten()



# # shuffle trials
N = N[:,list(randidx),:]
M = M[:,list(randidx),:]

N.shape

(250, 254, 10)

In [34]:
# reshape N and M to be (time*trials,factors/features)
N_reshape = N.reshape((N.shape[0]*N.shape[1],N.shape[2]))
M_reshape = M.reshape((M.shape[0]*M.shape[1],M.shape[2]))

In [35]:
bins_before = 20  # How many bins of neural data prior to the output are used for decoding
bins_current = 1  # Whether to use concurrent time bin of neural data
bins_after = 0    # How many bins of neural data after the output are used for decoding


# Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
# Function to get the covariate matrix that includes spike history from previous bins
X = get_spikes_with_history(N_reshape,bins_before,bins_after,bins_current) # (time*trials,nbins,factors)

# Format for  Dense Neural Network
# Put in "flat" format, so each "neuron / time" is a single feature
X_flat = X.reshape(X.shape[0],(X.shape[1]*X.shape[2])) # (time*trials, nbins*factors)

In [36]:
# Set decoding output
y = M_reshape

In [37]:
# Set what part of data should be part of the training/testing/validation sets
training_range = [0, 0.7]
testing_range = [0.7, 0.85]
valid_range = [0.85,1]

In [38]:
num_examples=X.shape[0]

# Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
# This makes it so that the different sets don't include overlapping neural data
training_set = np.arange(int(np.round(training_range[0]*num_examples))+bins_before,int(np.round(training_range[1]*num_examples))-bins_after)
testing_set = np.arange(int(np.round(testing_range[0]*num_examples))+bins_before,int(np.round(testing_range[1]*num_examples))-bins_after)
valid_set = np.arange(int(np.round(valid_range[0]*num_examples))+bins_before,int(np.round(valid_range[1]*num_examples))-bins_after)

# Get training data
X_train = X[training_set,:,:]
X_flat_train = X_flat[training_set,:]
y_train = y[training_set,:]

# Get testing data
X_test = X[testing_set,:,:]
X_flat_test = X_flat[testing_set,:]
y_test = y[testing_set,:]

# Get validation data
X_valid = X[valid_set,:,:]
X_flat_valid = X_flat[valid_set,:]
y_valid = y[valid_set,:]

In [39]:
# Z-score "X" inputs. 
X_train_mean = np.nanmean(X_train,axis=0)
X_train_std = np.nanstd(X_train,axis=0)
X_train = (X_train-X_train_mean)/X_train_std
X_test = (X_test-X_train_mean)/X_train_std
X_valid = (X_valid-X_train_mean)/X_train_std

# Z-score "X_flat" inputs. 
X_flat_train_mean = np.nanmean(X_flat_train,axis=0)
X_flat_train_std = np.nanstd(X_flat_train,axis=0)

X_flat_train = (X_flat_train-X_flat_train_mean)/X_flat_train_std 
X_flat_test = (X_flat_test-X_flat_train_mean)/X_flat_train_std 
X_flat_valid = (X_flat_valid-X_flat_train_mean)/X_flat_train_std 

#Zero-center outputs
y_train_mean = np.mean(y_train,axis=0)

y_train = y_train-y_train_mean 
y_test =  y_test-y_train_mean 
y_valid =  y_valid-y_train_mean 



In [42]:


def dnn_evaluate(units,dropout):
    model = DenseNNDecoder(units=units,dropout=dropout,num_epochs=20) #Define model
    model.fit(X_flat_train,y_train) #Fit model
    y_valid_predicted=model.predict(X_flat_valid) #Validation set predictions
    return np.mean(get_R2(y_valid,y_valid_predicted)) #R2 value of validation set (mean over x and y position/velocity)



In [43]:


#Define Bayesian optimization, and set limits of hyperparameters 
#Here, we set the limit of "degree" to be [1, 6.99], so we test degrees 1,2,3,4,5,6
dnnBO = BayesianOptimization(dnn_evaluate, {'units': (10, 399.99), 'dropout': (0,0.99)}, verbose=0)
#Set number of initial runs (init_points) and subsequent tests (n_iter), and do the optimization
#kappa is a parameter that sets exploration vs exploitation in the algorithm
#We set kappa=10 (greater than the default) so there is more exploration when there are more hyperparameters
dnnBO.maximize(init_points=5, n_iter=5, kappa=10) 



KeyboardInterrupt: 

In [21]:
best_dropout = dnnBO.max['params']['dropout']
best_units = dnnBO.max['params']['units']

In [41]:


#Declare model
# model_dnn=DenseNNDecoder(units=best_units,dropout=best_dropout,num_epochs=1000)
model_dnn=DenseNNDecoder(units=[400],dropout=0.1,num_epochs=20)

#Fit model
model_dnn.fit(X_flat_train,y_train)

#Get predictions
y_valid_predicted_dnn=model_dnn.predict(X_flat_valid)

#Get metric of fit
R2s_dnn=get_R2(y_valid,y_valid_predicted_dnn)
print('R2s:', R2s_dnn)



R2s: [-2.93062595e+02 -1.03886342e-01  3.72477907e-01 -1.58794681e-01
 -5.92395645e-02 -6.09336424e-02 -7.45069124e-02]
